# $Question_{1}$

## Include Neccessary libraries

In [45]:
import numpy as np
import pandas as pd

## Part 1

In [46]:
path = r'Data\training.1600000.processed.noemoticon.csv'
df = pd.read_csv(path, encoding='latin1')
df.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


### Add Header row to data!

In [47]:
columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
path = r'Data\training.1600000.processed.noemoticon.csv'
df = pd.read_csv(path, names=columns, header=None, encoding='latin1')
df.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


### 5000 Sample from each polarity(target)

In [48]:
sampled_df = df.groupby('target').apply(lambda x: x.sample(5000, random_state=42))
sampled_df = sampled_df.reset_index(drop=True)
sampled_df.head()

,target,ids,date,flag,user,text
0,0,1974671194,Sat May 30 13:36:31 PDT 2009,NO_QUERY,simba98,@xnausikaax oh no! where did u order from? tha...
1,0,1997882236,Mon Jun 01 17:37:11 PDT 2009,NO_QUERY,Seve76,A great hard training weekend is over. a coup...
2,0,2177756662,Mon Jun 15 06:39:05 PDT 2009,NO_QUERY,x__claireyy__x,"Right, off to work Only 5 hours to go until I..."
3,0,2216838047,Wed Jun 17 20:02:12 PDT 2009,NO_QUERY,Balasi,I am craving for japanese food
4,0,1880666283,Fri May 22 02:03:31 PDT 2009,NO_QUERY,djrickdawson,Jean Michel Jarre concert tomorrow gotta work...


In [49]:
print(len(sampled_df[sampled_df['target'] == 0]))
print(len(sampled_df[sampled_df['target'] == 4]))

5000
5000


## Import libraries for normalizing and tokenizer

In [50]:
import re
import string
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# Download this packages if neccessary!
import emoji

### A dictionary of emoticons!

In [51]:
emoticons_dict = {
    ":)": "happy",
    ":-)": "happy",
    ":]": "happy",
    ":D": "laughing",
    ":-D": "laughing",
    "XD": "laughing",
    "xD": "laughing",
    ":(": "sad",
    ":-(": "sad",
    ":[": "sad",
    ":'(": "crying",
    ":'-)": "tears_of_joy",
    ":'D": "tears_of_joy",
    ";)": "wink",
    ";-)": "wink",
    ";D": "wink",
    ">_<": "frustrated",
    ">.<": "frustrated",
    ":-/": "skeptical",
    ":/": "skeptical",
    ":\\": "skeptical",
    ":-|": "neutral",
    ":|": "neutral",
    ":-P": "playful",
    ":P": "playful",
    ":O": "surprised",
    ":-O": "surprised",
    ":-0": "shocked",
    ">:(": "angry",
    "D:<": "angry",
    "D-':": "angry",
    ":3": "cute",
    "O:)": "angelic",
    "3:)": "devilish",
    "<3": "love",
    "</3": "broken_heart",
    ":*": "kiss",
    ":-*": "kiss",
    ";*": "kiss",
    ":-S": "worried",
    "D:": "dismay",
    ":$": "embarrassed",
    ":-X": "sealed_lips",
    "8-)": "cool",
    "B-)": "cool",
    "-_-": "annoyed",
    "-__-": "annoyed",
    "-___-": "exhausted",
    ":^)": "nose_laugh",
    "|-O": "bored",
    ":@": "furious",
    ">:O": "upset",
    ":-)": "smiley",
    "XD": "big_grin",
    "^^": "joy",
    "^_^": "joy",
    ":-(": "frowny",
    "T_T": "cry",
    "Q_Q": "cry",
    "TT_TT": "cry",
    ">.<": "irritated",
    "-_-;": "sweat",
    "':(": "tear",
    "D;": "horrified",
    "o_O": "surprise",
    "O_o": "dismay",
    "o.o": "bewildered",
    "o_o": "stunned",
    "0:3": "innocent",
    "O:-)": "angel",
    "(>_<)": "trouble",
    "(^_^;)": "nervous",
    "(-_-;)": "worried",
    "(T_T)": "upset",
    "(-_-)zzz": "sleeping",
    "(^_^)/": "cheer",
    "(^-^)b": "thumbs_up",
    "(>_<)>": "frustrated",
    "(^o^)": "elated",
    "(-_-)": "annoyed",
}

acronyms_dict = {
    "lol": "laughing out loud",
    "btw": "by the way",
    "imho": "in my humble opinion",
    "fyi": "for your information",
    "brb": "be right back",
    "idk": "I don't know",
    "imo": "in my opinion",
    "irl": "in real life",
    "smh": "shaking my head",
    "tbh": "to be honest",
    "rofl": "rolling on the floor laughing",
    "bff": "best friends forever",
    "np": "no problem",
    "omg": "oh my god",
    "tmi": "too much information",
    "lmao": "laughing my ass off",
    "lmk": "let me know",
    "tbt": "throwback thursday",
    "jk": "just kidding",
    "icymi": "in case you missed it",
    "stfu": "shut the f*** up",
    "dm": "direct message",
    "nvm": "never mind",
    "ftw": "for the win",
    "wtf": "what the f***",
    "ftl": "for the loss",
    "afaik": "as far as I know",
    "ama": "ask me anything",
    "fomo": "fear of missing out",
    "yolo": "you only live once",
}

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

print(stop_words)

{'doing', "won't", 'their', 'herself', 'd', 'these', 'the', 'how', 'our', 'been', 'under', 'will', 'before', "it's", "couldn't", 't', 'i', 'that', 'its', 'do', 'shan', 'own', 'haven', 'those', 'my', "don't", 'ain', "doesn't", 'won', "weren't", 'she', 'if', 'hasn', 'are', 'his', 'other', 've', 'while', 'all', "mustn't", 'because', 'very', 'should', 'any', 'nor', 'about', 'yours', "you've", 'after', 'don', 'ma', 'a', 'themselves', 'into', 'again', 'more', 'so', "you're", 'where', 'myself', 'once', 'few', 'll', "wouldn't", "aren't", 'an', 'it', 'off', 'each', 'not', 'just', "shouldn't", 'doesn', 'during', 'but', 'he', 'down', "hadn't", 'what', 're', 'some', 'hadn', 'having', "that'll", 'theirs', 'this', 'then', 'until', 'does', 'now', 'yourselves', 'them', 'at', 'above', 'to', 'no', 'mightn', 'needn', 'isn', 'and', 'o', 'hers', 'same', 'you', 'or', 'ours', "she's", 'for', "hasn't", 'when', "didn't", 'against', 'as', 'wouldn', 'be', "shan't", "isn't", "haven't", 'did', 'has', 'himself', 'y

In [52]:
class preprocess:
    def __init__(self, emoticons_dict, stop_words, lemmatizer, acronyms_dict):
        self.text = None
        self.map = emoticons_dict
        self.stop_words_set = stop_words
        self.lemmatizer = lemmatizer
        self.accronyms = acronyms_dict
    
    def assign_text(self, text):
        self.text = text    

    def normalize_tokenize(self):
        self.text = self._lower_case()
        self.text = self._remove_user_mention()
        self.text = self._remove_URLs()
        self.text = self._remove_emalis()
        self.text = self._replace_emoticons()
        self.text = self._replace_emojies()
        self.text = self._remove_URLs()
        self.text = self._handling_hashtags()
        self.text = self._repeated_characters() # Feel free to rerun the code based on ignoring repeated charcters or not!
        self.text = self._remove_punctuations()
        self.text = self._remove_non_ascii_chars()
        self.text = self._remove_numbers()
        self.text = self._expand_accronyms()
        tokens = self._tokenizing()
        tokens = self._lemmatize_tokens(tokens)
        return tokens

    def _lower_case(self):
        return self.text.lower()
    
    def _remove_user_mention(self):
        return re.sub(r'@\w+', '', self.text)
    
    def _remove_URLs(self):
        return re.sub(r'http\S+|www\S+|https\S+', '', self.text, flags=re.MULTILINE)
    
    def _replace_emoticons(self):
        return " ".join(self.map.get(word, word) for word in self.text.split())
    
    def _replace_emojies(self):
        return emoji.demojize(self.text, delimiters=("",""))

    def _handling_hashtags(self):
        return self.text.replace('#', '')
    
    def _repeated_characters(self):
        return re.sub(r'(.)\1+', r'\1\1', self.text)

    def _remove_punctuations(self): # Except important one like !
        return self.text.translate(str.maketrans('', '', string.punctuation.replace('!', '')))

    def _remove_numbers(self):
        return re.sub(r'\b\d+(?:,\d+)*(?:\.\d+)?\b', '', self.text)

    def _remove_non_ascii_chars(self):
        return self.text.encode('ascii', 'ignore').decode('ascii')
    
    def _expand_accronyms(self):
        return " ".join(self.accronyms.get(word, word) for word in self.text.split())


    def _tokenizing(self):
        return word_tokenize(self.text)
    
    def _lemmatize_tokens(self, tokens):
        return [self.lemmatizer.lemmatize(token) for token in tokens]

    def _remove_emalis(self):
        email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        return re.sub(email_pattern, '', self.text)
    

In [53]:
preprocess_cls = preprocess(emoticons_dict, stop_words, lemmatizer, acronyms_dict)

### Give an example for see how good this preprocess class is?

In [54]:
example = """Hey @user123, can't wait for the meetup tonight! 🥳 #excited #networking 
OMG, did you see the latest episode of #TheTechShow? Mind-blowing theories there... 🤯💡 
LOL, my code passed all tests on first 12412342133 run - feel like a boss 😎👩‍💻
Btw, check out this cool article on quantum computing: https://bit.ly/3xyzABC
P.S. @competitor did a boo-boo again 🙈 #facepalm """

In [55]:
preprocess_cls.assign_text(example)
prep_text = preprocess_cls.normalize_tokenize()
print(prep_text)

['hey', 'cant', 'wait', 'for', 'the', 'meetup', 'tonight', '!', 'partyingface', 'excited', 'networking', 'oh', 'my', 'god', 'did', 'you', 'see', 'the', 'latest', 'episode', 'of', 'thetechshow', 'mindblowing', 'theory', 'there', 'explodingheadlightbulb', 'laughing', 'out', 'loud', 'my', 'code', 'passed', 'all', 'test', 'on', 'first', 'run', 'feel', 'like', 'a', 'bos', 'smilingfacewithsunglasseswomantechnologist', 'by', 'the', 'way', 'check', 'out', 'this', 'cool', 'article', 'on', 'quantum', 'computing', 'p', 'did', 'a', 'booboo', 'again', 'seenoevilmonkey', 'facepalm']


In [56]:
def preprocess_text(text):
    preprocess_cls.assign_text(text)
    return preprocess_cls.normalize_tokenize()

sampled_df['tokens'] = sampled_df['text'].apply(preprocess_text)

In [57]:
sampled_df.tokens

0       [oh, no, !, where, did, u, order, from, thats,...
1       [a, great, hard, training, weekend, is, over, ...
2       [right, off, to, work, only, hour, to, go, unt...
3                   [i, am, craving, for, japanese, food]
4       [jean, michel, jarre, concert, tomorrow, got, ...
                              ...                        
9995                             [love, to, see, that, !]
9996    [lovely, day, spent, with, the, miss, just, lo...
9997    [i, love, the, fact, that, people, are, essent...
9998    [time, to, shower, and, tidy, then, going, to,...
9999               [and, it, an, excuse, to, get, a, wii]
Name: tokens, Length: 10000, dtype: object

### Import sklearn for spliting

In [58]:
from sklearn.model_selection import train_test_split

### Split equaly  between classes

In [59]:
negative_class = sampled_df[sampled_df['target'] == 0]
positive_class = sampled_df[sampled_df['target'] == 4]

In [60]:
X_pos_train, X_pos_test, y_pos_train, y_pos_test = train_test_split(positive_class['tokens'], positive_class['target'], test_size=0.2, random_state=2)
X_neg_train, X_neg_test, y_neg_train, y_neg_test = train_test_split(negative_class['tokens'], negative_class['target'], test_size=0.2, random_state=2)

### Concatenate them

In [61]:
X_train = pd.concat([X_pos_train, X_neg_train])

In [62]:
X_test = pd.concat([X_pos_test, X_neg_test])

In [63]:
y_train = pd.concat([y_pos_train, y_neg_train])

In [64]:
y_test = pd.concat([y_pos_test, y_neg_test])

In [65]:
print(y_test[y_test==0].shape, y_test[y_test==4].shape)

(1000,) (1000,)


## Part 2

### Vocab set

In [66]:
vocab_set = {}
all_tokens = []
for index,tokens in X_train.items():
    all_tokens.extend(tokens)

print(len(all_tokens))
vocab_set = set(all_tokens)
print(len(vocab_set))

104608
11600


### Create a fucntion for Term Frequency(TF)

In [67]:
class freq_matrix:
    def __init__(self, X_train, X_test, vocab_set):
        self.train = X_train
        self.test = X_test
        self.vocab_set = vocab_set
        self.vocab_set.add('OOV')
        print(len(self.vocab_set))
        self.unique_words_index = list(self.vocab_set)
    
    def term_frequency_matrix_train(self):
        real_index = 0
        tf_matrix = np.zeros(shape=(len(self.train), len(self.vocab_set)), dtype=float)
        for _,tokens in self.train.items():
            for token in tokens:
                word_index = self.unique_words_index.index(token)
                tf_matrix[real_index][word_index] += 1
            word_index = self.unique_words_index.index('OOV')
            tf_matrix[real_index][word_index] = np.mean(tf_matrix[real_index])
            real_index += 1
        tf_matrix = tf_matrix + 1
        return np.where(tf_matrix > 1, 1 + np.log(tf_matrix), 0), tf_matrix
    
    def term_frequncy_matrix_test(self):
        real_index = 0
        unseen_data_set = set()
        tf_matrix = np.zeros(shape=(len(self.test), len(self.vocab_set)), dtype=float)
        for _,tokens in self.test.items():
            for token in tokens:
                if token in self.vocab_set:
                    word_index = self.unique_words_index.index(token)
                    tf_matrix[real_index][word_index] += 1
                else:
                    unseen_data_set.add(token)
                    word_index = self.unique_words_index.index('OOV')
                    tf_matrix[real_index][word_index] += 1
            real_index += 1
        tf_matrix = tf_matrix + 1
        return np.where(tf_matrix > 1, 1 + np.log(tf_matrix), 0), tf_matrix, unseen_data_set


In [68]:
my_freq_matrix = freq_matrix(X_train, X_test, vocab_set)

11601


In [69]:
log_train_tf, ordinary_tf_Train = my_freq_matrix.term_frequency_matrix_train()
log_train_tf.shape

(8000, 11601)

In [70]:
log_test_tf, ordinary_tf_test, unseen_data = my_freq_matrix.term_frequncy_matrix_test()
log_test_tf.shape

(2000, 11601)

In [71]:
print(len(unseen_data))
print(len(unseen_data)/len(vocab_set))

1786
0.15395224549607792


## Part 3

In [72]:
def tf_idf_matrix_train(self, log_tf, train_tf):
        train_tf_idf = np.zeros(shape=(log_tf.shape))
        term_freq = np.copy(train_tf)
        df = np.zeros(shape=(train_tf.shape[1]))
        idf = np.zeros(shape=(train_tf.shape[1]))
        for i,_ in enumerate(self.unique_words_index):
                df[i] = (term_freq[:,i] > 1).sum()
                idf[i] = np.log(train_tf.shape[0]/(df[i] + 1))
        idf[self.unique_words_index.index('OOV')] = np.log(train_tf.shape[0]/1)
        for i in range(len(self.unique_words_index)):
                train_tf_idf[:,i] = log_tf[:, i] * idf[i]
        return train_tf_idf

def tf_idf_matrix_test(self, log_tf, test_tf):
        train_tf_idf = np.zeros(shape=(log_tf.shape))
        term_freq = np.copy(test_tf)
        df = np.zeros(shape=(test_tf.shape[1]))
        idf = np.zeros(shape=(test_tf.shape[1]))
        for i,_ in enumerate(self.unique_words_index):
                df[i] = (term_freq[:,i] > 1).sum()
                idf[i] = np.log(test_tf.shape[0]/(df[i] + 1))
        for i in range(len(self.unique_words_index)):
                train_tf_idf[:,i] = log_tf[:, i] * idf[i]
        return train_tf_idf

freq_matrix.tf_idf_matrix_train = tf_idf_matrix_train
freq_matrix.tf_idf_matrix_test = tf_idf_matrix_test

In [73]:
tf_idf_train = my_freq_matrix.tf_idf_matrix_train(log_train_tf, ordinary_tf_Train)
tf_idf_train.shape

(8000, 11601)

In [74]:
tf_idf_test = my_freq_matrix.tf_idf_matrix_test(log_test_tf, ordinary_tf_test)
tf_idf_test.shape

(2000, 11601)

### Part 4

### Create Methods to calculate PPMI for word-word 

In [75]:
from nltk import ngrams, FreqDist
def bigram_freq(self):
    all_tokens = []
    for _, tokens in self.train.items():
        all_tokens.extend(tokens)
    return FreqDist(list(ngrams(all_tokens, 2)))

def co_occurence_matrix(self):
    co_matrix = np.zeros(shape=(len(self.vocab_set), len(self.vocab_set)))
    bigrams = self.bigram_freq()
    for bigram,value in bigrams.items():
        word_index1 = self.unique_words_index.index(bigram[0])
        word_index2 = self.unique_words_index.index(bigram[1])
        co_matrix[word_index1, word_index2] += value
    word_index = self.unique_words_index.index('OOV')
    co_matrix[word_index, :] = np.mean(co_matrix, axis=0)
    co_matrix[:, word_index] = np.mean(co_matrix, axis=1)
    co_matrix += 1
    return co_matrix

def ppmi_matrix(self, occurence_matrix):
    co_matrix = np.copy(occurence_matrix)
    total_sums = np.sum(co_matrix)
    co_matrix = co_matrix / total_sums
    p_rows = np.sum(co_matrix, axis=1) / total_sums
    p_cols = np.sum(co_matrix, axis=0) / total_sums
    pmi_matrix = np.log2(np.divide(co_matrix, np.outer(p_rows, p_cols)))
    ppmi_matrix = np.maximum(pmi_matrix, 0)
    return ppmi_matrix



freq_matrix.bigram_freq = bigram_freq
freq_matrix.co_occurence_matrix = co_occurence_matrix
freq_matrix.ppmi_matrix = ppmi_matrix

In [76]:
occurence_matrix = my_freq_matrix.co_occurence_matrix()
ppmi_matrix = my_freq_matrix.ppmi_matrix(occurence_matrix)

### Embedding the documents based on our vectors

In [77]:
def doc_embeding_train(self, word_ppmi):
    doc_word_ppmi = np.zeros(shape=(len(self.train), len(self.vocab_set)), dtype=float)
    real_index = 0
    for _,tokens in self.train.items():
        counter = 0
        for token in tokens:
            doc_word_ppmi[real_index] += word_ppmi[self.unique_words_index.index(token), :]
            counter += 1
        if counter != 0:
            doc_word_ppmi[real_index] = doc_word_ppmi[real_index] / counter
        real_index += 1
    return doc_word_ppmi
        
def doc_embedding_test(self, word_ppmi):
    doc_word_ppmi = np.zeros(shape=(len(self.test), len(self.vocab_set)), dtype=float)
    real_index = 0
    for _,tokens in self.test.items():
        counter = 0
        for token in tokens:
            if token in self.vocab_set:
                doc_word_ppmi[real_index] += word_ppmi[self.unique_words_index.index(token), :]
            else:
                doc_word_ppmi[real_index] += word_ppmi[self.unique_words_index.index('OOV'), :]
            counter += 1
        if counter!=0:
            doc_word_ppmi[real_index] = doc_word_ppmi[real_index] / counter
        real_index += 1
    return doc_word_ppmi

freq_matrix.doc_embeding_train = doc_embeding_train
freq_matrix.doc_embedding_test = doc_embedding_test

In [78]:
train_doc_ppmi = my_freq_matrix.doc_embeding_train(ppmi_matrix)
train_doc_ppmi.shape

(8000, 11601)

In [79]:
test_doc_ppmi = my_freq_matrix.doc_embedding_test(ppmi_matrix)
test_doc_ppmi.shape

(2000, 11601)

## Part 5

In [80]:
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [81]:
clf = MultinomialNB()
clf.fit(log_train_tf, y_train)

MultinomialNB()

In [82]:
y_pred = clf.predict(log_test_tf)

In [83]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, pos_label=4))
print(precision_score(y_test, y_pred, pos_label=4))
print(f1_score(y_test, y_pred, pos_label=4))

0.7305
0.686
0.7530186608122942
0.7179487179487181


In [84]:
clf = MultinomialNB()
clf.fit(tf_idf_train, y_train)

MultinomialNB()

In [85]:
y_pred = clf.predict(tf_idf_test)
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, pos_label=4))
print(precision_score(y_test, y_pred, pos_label=4))
print(f1_score(y_test, y_pred, pos_label=4))

0.695
0.651
0.7138157894736842
0.6809623430962343


In [86]:
clf = MultinomialNB()
clf.fit(train_doc_ppmi, y_train)

MultinomialNB()

In [87]:
y_pred = clf.predict(test_doc_ppmi)

In [88]:
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred, pos_label=4))
print(precision_score(y_test, y_pred, pos_label=4))
print(f1_score(y_test, y_pred, pos_label=4))

0.591
0.478
0.6175710594315246
0.5388951521984217
